In [18]:
import time
import cv2
import math

import numpy as np

from swarm import Swarm


In [19]:
NUM_DRONES = 3
GRID_SIZE = 50           # Size of the Forest
grid_offset=GRID_SIZE/2

MOVE_DISTANCE = 1       # How far a Drone can move each Timestep
TIMESTEPS = 10
POPULATION_SIZE = 50
NUM_GENERATIONS = 10
MUTATION_RATE = 0.1

AREA_SIZE_X = 50
AREA_SIZE_Y = 50
INITIAL_VISIBILITY = 0.5

In [17]:
#return percentage of ground visible (depth>34meter)
def check_ground_visibility(depth_image):
    number_visible_ground=0
    for i in depth_image:
        for j in i:
            if j[1]>3400:
                number_visible_ground= number_visible_ground+1
    return number_visible_ground/(512**2)
 
def update_Target_Position():
    if Target_Position[0]<25:
        Target_Position = Target_Position + [0.5,0.5]

In [9]:
#def mutate(solution):
    # Waypints in Solution get randomized
def crossover(parent1, parent2):
    crossover_point = random.randint(0, TIMESTEPS - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

Initialization of Drones and Solution Populations

In [20]:
#Spawn Drones and move to initial position

# Create the swarm object by passing the name
# of the world from the .sdf world file.
swarm = Swarm("rhea_swarm")

# Spawn X drones and keep the returning ids as handles
ids = swarm.spawn(NUM_DRONES)

#Create Fake Person Location
person_waypoint =np.array([5,5])

# First waypoints
waypoints = np.empty((0,3),float)
for i in range (NUM_DRONES):
    waypoints= np.append(waypoints,np.array([[math.ceil(i-(NUM_DRONES/2)),0.0,35.0]]), axis=0)

#create Initial
swarm.waypoints(ids, waypoints)

#Visibility by Coordiantes File:
grid_visibility = np.full((AREA_SIZE_X,AREA_SIZE_Y), INITIAL_VISIBILITY)

#Initialize Target Position
Target_Position = [10,10]

Last_Known_Position = Target_Position

Subscribed successfully


In [13]:
#print(swarm.depth_images[1])
print(check_ground_visibility(swarm.depth_images[1]))


0.7034645080566406


In [8]:
for d_z in range(0, 30, 5):

    # Set the waypoints for all drones
    current_waypoints = waypoints
    swarm.waypoints(ids, current_waypoints)

    # Wait until the data has arrived
    time_delta = 0.01           # Delta time per sleep command in seconds
    time_passed = 0.0           # Time counter to keep track of the time in seconds
    timeout = 1.0               # Timeout in case something goes wrong

    timeout_occured = False

    while time_passed < timeout:
        # All frames for a waypoint called send
        # together, so it is enough to check the
        # last one.
        if swarm.received_frames[ids[-1]]:
            print("All frames recieved")
            
            print("depth image for ID={}:".format(1))
            d_i = swarm.depth_images[1]
            print(np.shape(d_i))
            #for i in range(len(d_i)):
            #    print(d_i[i])
            
            for id in ids:
                rgb_image = swarm.rgb_images[id]
                #thermal_image = swarm.thermal_images[id]
                depth_image = swarm.depth_images[id]
                # In this example we only store them in a data directory
                # OpenCV expects BGR but it is RGB, so switch the channels
                c#v2.imwrite(f"../../data/swarm/rgb_image_drone_{id}_dz_{d_z}.png", cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB))    
                #cv2.imwrite(f"../../data/swarm/thermal_image_drone_{id}_dz_{d_z}.png", thermal_image)
                #cv2.imwrite(f"../../data/swarm/depth_image_drone_{id}_dz_{d_z}.png", depth_image)
            break
            
            
            
        time.sleep(time_delta)
        time_passed += time_delta
    if time_passed >= timeout:
            timeout_occured = True
            print(f"Timeout reached for waypoints with dz={d_z}")

if timeout_occured:
    raise TimeoutError("Timeout occured while waiting for waypoint")
    
print("Finished")

Timeout reached for waypoints with dz=0
Timeout reached for waypoints with dz=5
Timeout reached for waypoints with dz=10
Timeout reached for waypoints with dz=15
Timeout reached for waypoints with dz=20
Timeout reached for waypoints with dz=25


TimeoutError: Timeout occured while waiting for waypoint

In [14]:
# Problem classes
from pymoo.core.problem import Problem, ElementwiseProblem
from pymoo.problems.functional import FunctionalProblem
from pymoo.optimize import minimize

# Algorithmss
from pymoode.algorithms import DE, GDE3, NSDE, NSDER
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.nsga2 import NSGA2

# Reference directions and test problems
from pymoo.problems import get_problem
from pymoo.util.ref_dirs import get_reference_directions

# Survival operators
from pymoode.survival import RankAndCrowding, ConstrRankAndCrowding

# Termination
from pymoo.termination.default import DefaultSingleObjectiveTermination, DefaultMultiObjectiveTermination

# Performance
from pymoode.performance import SpacingIndicator


In [15]:
class MyProblem(Problem):

    def __init__(self, grid_size, n_drones, n_steps, grid_visibility, target_probabilities):
        
        self.grid_size = grid_size
        self.n_drones = n_drones
        self.n_steps = n_steps
        self.grid_visibility = grid_visibility
        self.target_probabilities = target_probabilities

        super().__init__(
            n_var=2, # Variables in the solutions (x,y)
            n_obj=2, # Number of Objectives we wanna optimize
            n_constr=0,  # No additional constraints, TODO: add no crashing into eachother
            xl=-2.0,   #Lower boundaries of solution variables
            xu=2.0)    #upper bounds of solution vars

        #x are the waypoints
    def _evaluate(self, x, out, *args, **kwargs):
        
        x_cord=int(x[0]+grid_offset)
        y_cord=int((x[1]+grid_offset)
                   
        #TODO:Probability of Target, currently just distance to target
        probability = dist = numpy.linalg.norm(x-Target_Position)
                   
        #Visibility of Ground: Percentage of Ground Visible from that position.
        visibility = grid_visibility[x_cord][y_cord]
        
        
        out["F"] = np.column_stack([-visibility, -probability])


# Initialize parameters
target_probabilities = np.random.rand(AREA_SIZE_X, AREA_SIZE_Y)

# Define the problem
problem = MyProblem(AREA_SIZE_X, NUM_DRONES, NUM_GENERATIONS, grid_visibility, target_probabilities)

# Define the algorithm
algorithm = DE(
    pop_size=50,
    sampling="real_random",
    variant="DE/rand/1/bin",
    crossover=get_crossover("int_sbx", prob=0.9, eta=15),
    mutation=get_mutation("int_pm", eta=20),
    eliminate_duplicates=True
)



termination = get_termination("n_gen", 200)

result = minimize(
    problem,
    algorithm, 
    termination, 
    seed=1, 
    save_history=True, 
    verbose=True)

best_solution = result.X[np.argmin(result.F[:, 0] + result.F[:, 1])]

NameError: name 'initial_coverage' is not defined

In [ ]:
finished=False
#Drone waypoints got from -25 to 25, Array from 0 to 50

while(!finished):
    
    #wait until all drones recieved
    if swarm.received_frames[ids[-1]]:
        for id in ids:
            drone_waypoint=waypoints[id]
            #rgb_image = swarm.rgb_images[id]
            vis = check_ground_visibility(swarm.depth_images[id])
            
            x_cord=int(drone_waypoint[0]+grid_offset)
            y_cord=int((drone_waypoint[1]+grid_offset)
            
            grid_visibility[x_cord][y_cord]=vis
            #TODO: Check if target was seen, update last known position
        break
        
        #Find new best waypoints
        best_solution = result.X
        best_fitness = result.F
        print("Best Solution: {} \n with a Score of {}".format(best_solution,best_fitness))
        
        #Move Drones to best waypoints
        current_waypoints = best_solution
        swarm.waypoints(ids, current_waypoints)
        
        #Move Target-Person
        print("Moving Target to {}")